# 병합 기준 열을 여러 개 사용하기
--- 
## 데이터
- NLS에서 추출한 2000~2004년 근무 주 수 파일과 대학 등록 파일
- 한 행에 한 명에 대한 1년간의 정보

In [4]:
import pandas as pd
nls97weeksworked = pd.read_csv('./data/nls97weeksworked.csv')
nls97colenr = pd.read_csv('./data/nls97colenr.csv')

In [5]:
# 근무주 수 데이터 살피기
nls97weeksworked.sample(10, random_state = 1)

,originalid,year,weeksworked
32923,7199,2003,0.0
14214,4930,2001,52.0
2863,4727,2000,13.0
9746,6502,2001,0.0
2479,4036,2000,28.0
39435,1247,2004,52.0
36416,3481,2004,52.0
6145,8892,2000,19.0
5348,8411,2000,0.0
24193,4371,2002,34.0


In [6]:
nls97weeksworked.shape

(44920, 3)

In [7]:
nls97weeksworked.originalid.nunique()

8984

In [8]:
nls97colenr.sample(10, random_state=1)

,originalid,year,colenr
32923,7199,2003,1. Not enrolled
14214,4930,2001,1. Not enrolled
2863,4727,2000,NaN
9746,6502,2001,1. Not enrolled
2479,4036,2000,1. Not enrolled
39435,1247,2004,3. 4-year college
36416,3481,2004,1. Not enrolled
6145,8892,2000,1. Not enrolled
5348,8411,2000,1. Not enrolled
24193,4371,2002,2. 2-year college


In [9]:
nls97colenr.shape

(44920, 3)

In [11]:
nls97colenr.originalid.nunique()

8984

### 병합 기준 열의 유일값 확인

In [12]:
nls97weeksworked.groupby(['originalid', 'year'])['originalid'].count().shape

(44920,)

In [15]:
nls97colenr.groupby(['originalid', 'year'])['originalid'].count().shape

(44920,)

### 병합 기준 열에 불일치 있는지 확인

In [33]:
def checkmerge(dfleft, dfright, idvar):
    dfleft['inleft'] = "Y"
    dfright['inright'] = "Y"
    dfboth = pd.merge(dfleft[idvar + ['inleft']], dfright[idvar, ['inright']], on=idvar, how='outer')
    dfboth.fillna('N', inplace = True)
    print(pd.crosstab(dfboth.inleft, dfboth.inright))

In [34]:
checkmerge(nls97weeksworked.copy(), nls97colenr.copy(), ['originalid', 'year'])

InvalidIndexError: (['originalid', 'year'], ['inright'])

### 복수의 병합 기준 열을 가지고 병합 수행

In [35]:
nlsworkschool = pd.merge(nls97weeksworked, nls97colenr, on=['originalid', 'year'], how='inner')

In [36]:
nlsworkschool.sample(10, random_state=1)

,originalid,year,weeksworked,inleft,colenr,inright
32923,7199,2003,0.0,Y,1. Not enrolled,Y
14214,4930,2001,52.0,Y,1. Not enrolled,Y
2863,4727,2000,13.0,Y,NaN,Y
9746,6502,2001,0.0,Y,1. Not enrolled,Y
2479,4036,2000,28.0,Y,1. Not enrolled,Y
39435,1247,2004,52.0,Y,3. 4-year college,Y
36416,3481,2004,52.0,Y,1. Not enrolled,Y
6145,8892,2000,19.0,Y,1. Not enrolled,Y
5348,8411,2000,0.0,Y,1. Not enrolled,Y
24193,4371,2002,34.0,Y,2. 2-year college,Y
